In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# GPU Support available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
num_epochs =4
batch_size =4
learning_rate = 0.001

In [5]:
# loading dataset - cifar-10
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)



100%|██████████| 170M/170M [00:03<00:00, 56.1MB/s]


In [6]:
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [12]:
# architecture
class convnet(nn.Module):
      def __init__(self):
        super(convnet,self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

      def forward(self,x):
        x= self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [13]:
model = convnet().to(device)


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [15]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
         # origin shape - [4,3,32,32] = 4,3,1024
         # input layer - 3 input channels, 6 output channels, 5 kernel size

         images = images.to(device)
         labels = labels.to(device)

         # Forward pass
         outputs = model(images)
         loss = criterion(outputs, labels)

         # Backward and optimize
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()

         if (i+1) % 2000 == 0:
             print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/4], Step [2000/12500], Loss: 2.2883
Epoch [1/4], Step [4000/12500], Loss: 2.3196
Epoch [1/4], Step [6000/12500], Loss: 2.3141
Epoch [1/4], Step [8000/12500], Loss: 2.2671
Epoch [1/4], Step [10000/12500], Loss: 2.4186
Epoch [1/4], Step [12000/12500], Loss: 1.9788
Epoch [2/4], Step [2000/12500], Loss: 1.7169
Epoch [2/4], Step [4000/12500], Loss: 2.2632
Epoch [2/4], Step [6000/12500], Loss: 2.1747
Epoch [2/4], Step [8000/12500], Loss: 1.6876
Epoch [2/4], Step [10000/12500], Loss: 1.0620
Epoch [2/4], Step [12000/12500], Loss: 0.9174
Epoch [3/4], Step [2000/12500], Loss: 3.0118
Epoch [3/4], Step [4000/12500], Loss: 1.7608
Epoch [3/4], Step [6000/12500], Loss: 3.7613
Epoch [3/4], Step [8000/12500], Loss: 1.6375
Epoch [3/4], Step [10000/12500], Loss: 1.0521
Epoch [3/4], Step [12000/12500], Loss: 1.3018
Epoch [4/4], Step [2000/12500], Loss: 1.5809
Epoch [4/4], Step [4000/12500], Loss: 1.6390
Epoch [4/4], Step [6000/12500], Loss: 1.6400
Epoch [4/4], Step [8000/12500], Loss: 1.7551
Epoc

In [18]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print("Accuracy -> ", acc)

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy ->  46.9
Accuracy of plane: 49.9 %
Accuracy of car: 60.3 %
Accuracy of bird: 20.7 %
Accuracy of cat: 30.0 %
Accuracy of deer: 36.7 %
Accuracy of dog: 44.6 %
Accuracy of frog: 68.3 %
Accuracy of horse: 54.7 %
Accuracy of ship: 53.8 %
Accuracy of truck: 50.0 %
